# Advanced Customer Service Agent with LangChain

This notebook implements a multi-component customer service agent using LangChain Expression Language (LCEL), structured outputs with Pydantic, and LangSmith for observability.

## Table of Contents
1. [Setup and Configuration](#setup)
2. [Pydantic Models Definition](#models)
3. [Component 1: Query Analysis & Classification](#component1)
4. [Component 2: Dynamic Response Generation](#component2)
5. [Component 3: Conversation Summarization](#component3)
6. [Complete Chain with LCEL](#chain)
7. [Testing with Sample Queries](#testing)

## 1. Setup and Configuration <a id='setup'></a>

First, let's install the required dependencies and set up our environment.

In [ ]:
# Install required packages
# Uncomment the following line to install dependencies
# !pip install langchain langchain-openai pydantic python-dotenv

In [ ]:
# Import necessary libraries
import os
import json
from typing import Literal, Optional, List, Dict, Any
from datetime import datetime
from pydantic import BaseModel, Field

# LangChain imports
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_core.output_parsers import PydanticOutputParser

# Environment setup
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

print("✅ Libraries imported successfully!")

## 2. Pydantic Models Definition <a id='models'></a>

We'll define structured data models using Pydantic for:
- Query analysis and classification
- Entity extraction
- Conversation summaries

In [ ]:
# Define the ExtractedEntities model for capturing key information from queries
class ExtractedEntities(BaseModel):
    """Entities extracted from customer queries"""
    product_name: Optional[str] = Field(
        None, 
        description="The specific product mentioned by the user"
    )
    order_number: Optional[str] = Field(
        None, 
        description="The order number mentioned by the user (e.g., TEC-2024001)"
    )
    date: Optional[str] = Field(
        None, 
        description="Any date mentioned in the query"
    )

# Define the QueryAnalysis model for structured query classification
class QueryAnalysis(BaseModel):
    """Analyzes and classifies a customer query"""
    query_category: Literal[
        "technical_support", 
        "billing", 
        "returns", 
        "product_inquiry", 
        "general_information"
    ] = Field(
        description="Category of the customer query"
    )
    urgency_level: Literal["low", "medium", "high"] = Field(
        description="Urgency level of the query based on customer language and needs"
    )
    customer_sentiment: Literal["positive", "neutral", "negative"] = Field(
        description="Detected customer sentiment from their message"
    )
    entities: ExtractedEntities = Field(
        description="Key entities extracted from the query"
    )

print("✅ Query analysis models defined!")

In [ ]:
# Define the ConversationSummary model for logging interactions
class ConversationSummary(BaseModel):
    """A structured summary of the customer service interaction"""

    timestamp: str = Field(
        description="Timestamp of the interaction in ISO format"
    )
    customer_id: str = Field(
        default="auto_generated", 
        description="Customer identifier"
    )
    conversation_summary: str = Field(
        description="A concise, one-sentence summary of the interaction"
    )
    query_category: str = Field(
        description="Category of the customer query"
    )
    customer_sentiment: str = Field(
        description="Customer sentiment during the interaction"
    )
    urgency_level: str = Field(
        description="Urgency level of the query"
    )
    mentioned_products: List[str] = Field(
        description="List of products mentioned in the conversation"
    )
    extracted_information: ExtractedEntities = Field(
        description="Key entities extracted from the conversation"
    )
    resolution_status: Literal["resolved", "pending", "escalated"] = Field(
        description="Current status of the query resolution"
    )
    actions_taken: List[str] = Field(
        description="List of actions the agent took or suggested"
    )
    follow_up_required: bool = Field(
        description="Whether follow-up is required for this interaction"
    )

print("✅ Conversation summary model defined!")

## 3. Component 1: Query Analysis & Classification <a id='component1'></a>

This component analyzes incoming customer queries and extracts structured information.

```mermaid
flowchart LR
    subgraph "Input"
        A[Raw Query Text]
    end
    
    subgraph "Query Analyzer"
        B[Analysis Prompt]
        C[LLM with Structured Output]
        D[Pydantic Validation]
    end
    
    subgraph "Structured Output"
        E[QueryAnalysis Object]
    end
    
    A --> B
    B --> C
    C --> D
    D --> E
```

In [ ]:
# Initialize the Large Language Model
llm = ChatOpenAI(
    model="gpt-4.1",
    temperature=0.7,  # Balanced between creativity and consistency
)

# Create structured output versions of the model
# This ensures the model returns data in our Pydantic model format
llm_analyzer = llm.with_structured_output(QueryAnalysis)
llm_summarizer = llm.with_structured_output(ConversationSummary)

print("✅ LLM models initialized with structured output!")

In [ ]:
# Create the prompt template for query analysis
analysis_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are an AI assistant expert in analyzing customer service queries for TechStore Plus.
    
    TechStore Plus is an e-commerce technology store based in New York, USA.
    We sell electronics, provide technical support, handle warranties, offer financing, and accept trade-ins.
    
    Analyze the customer query and extract the following information:
    1. Query category (technical_support, billing, returns, product_inquiry, general_information)
    2. Urgency level (low, medium, high)
       - High: Emergency, urgent need, work-critical issues
       - Medium: Important but not immediately critical
       - Low: General inquiries, non-urgent matters
    3. Customer sentiment (positive, neutral, negative)
       - Positive: Happy, satisfied, grateful
       - Neutral: Matter-of-fact, professional
       - Negative: Frustrated, angry, disappointed
    4. Key entities like product names, order numbers (format: TEC-YYYYNNN), dates
    
    Be precise and accurate in your analysis."""),
    ("human", "{query}")
])

# Create the query analyzer by chaining the prompt with the structured LLM
query_analyzer = analysis_prompt | llm_analyzer

print("✅ Query analyzer component created!")

In [ ]:
# Test the query analyzer with a sample query
test_query = "This is an emergency! My order #TEC-2024001 never arrived and I need that laptop for work tomorrow!"

print("🧪 Testing Query Analyzer")
print(f"Query: {test_query}\n")

analysis_result = query_analyzer.invoke({"query": test_query})
print("Analysis Result:")
print(f"- Category: {analysis_result.query_category}")
print(f"- Urgency: {analysis_result.urgency_level}")
print(f"- Sentiment: {analysis_result.customer_sentiment}")
print(f"- Entities: {analysis_result.entities}")

## 4. Component 2: Dynamic Response Generation <a id='component2'></a>

This component generates context-aware responses based on the query analysis.

```mermaid
graph TD
    A[Query Analysis Result]
    B{Route by Category}
    
    C[Technical Support Prompt]
    D[Billing Prompt]
    E[Returns Prompt]
    F[Product Inquiry Prompt]
    G[General Info Prompt]
    
    H[LLM Generation]
    I[Personalized Response]
    
    A --> B
    B -->|technical_support| C
    B -->|billing| D
    B -->|returns| E
    B -->|product_inquiry| F
    B -->|general_information| G
    
    C --> H
    D --> H
    E --> H
    F --> H
    G --> H
    
    H --> I
```

In [ ]:
# Define specialized prompts for each query category

# Technical Support Prompt - Empathetic and solution-focused
technical_support_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are an empathetic technical support agent for TechStore Plus.
    
    Customer Analysis:
    - Sentiment: {customer_sentiment}
    - Urgency Level: {urgency_level}
    - Product Mentioned: {product_name}
    
    Guidelines:
    - Be especially empathetic if the customer is frustrated
    - Provide clear, step-by-step troubleshooting instructions
    - Acknowledge their frustration and urgency
    - Offer immediate solutions or escalation paths
    - Keep responses concise but thorough"""),
    ("human", "{original_query}")
])

# Billing Prompt - Professional and precise
billing_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a professional billing agent for TechStore Plus.
    
    Customer Analysis:
    - Sentiment: {customer_sentiment}
    - Order Number: {order_number}
    - Date Mentioned: {date}
    
    Guidelines:
    - Be professional and accurate with billing information
    - Reference specific order numbers when mentioned
    - Explain billing policies clearly
    - Offer to email receipts or documentation
    - For urgent matters, prioritize quick resolution"""),
    ("human", "{original_query}")
])

# Returns Prompt - Understanding and clear about policies
returns_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a understanding returns specialist for TechStore Plus.
    
    Customer Analysis:
    - Sentiment: {customer_sentiment}
    - Product: {product_name}
    - Order Number: {order_number}
    
    Return Policy:
    - 30-day return window with original receipt
    - Products must be in original condition
    - Refunds processed within 5-7 business days
    
    Guidelines:
    - Be understanding of customer concerns
    - Clearly explain the return process
    - Offer prepaid return labels for defective items
    - Mention warranty options if applicable"""),
    ("human", "{original_query}")
])

# Product Inquiry Prompt - Enthusiastic and informative
product_inquiry_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are an enthusiastic product advisor for TechStore Plus.
    
    Customer Analysis:
    - Sentiment: {customer_sentiment}
    - Product Inquired: {product_name}
    
    Guidelines:
    - Be enthusiastic and helpful about products
    - Provide detailed product information
    - Mention availability and shipping times
    - Suggest related products or accessories
    - Include pricing information when relevant"""),
    ("human", "{original_query}")
])

# General Information Prompt - Friendly and comprehensive
general_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a friendly customer service agent for TechStore Plus.
    
    Customer Analysis:
    - Sentiment: {customer_sentiment}
    
    Guidelines:
    - Be friendly and professional
    - Provide comprehensive information
    - Direct customers to appropriate resources
    - Maintain a helpful tone throughout"""),
    ("human", "{original_query}")
])

print("✅ All category-specific prompts created!")

In [ ]:
# Create a routing dictionary to map categories to their respective prompts
prompt_router = {
    "technical_support": technical_support_prompt,
    "billing": billing_prompt,
    "returns": returns_prompt,
    "product_inquiry": product_inquiry_prompt,
    "general_information": general_prompt
}

# Define the routing function that selects the appropriate prompt
def route_to_prompt(data: Dict[str, Any]) -> Dict[str, Any]:
    """
    Routes the query to the appropriate prompt based on its category.
    
    Args:
        data: Dictionary containing 'analysis' and 'original_query'
        
    Returns:
        Dictionary with analysis, query, and formatted prompt
    """
    analysis = data["analysis"]
    category = analysis.query_category
    
    # Prepare data for the prompt template
    prompt_data = {
        "original_query": data["original_query"],
        "customer_sentiment": analysis.customer_sentiment,
        "urgency_level": analysis.urgency_level,
        "product_name": analysis.entities.product_name or "Not specified",
        "order_number": analysis.entities.order_number or "Not specified",
        "date": analysis.entities.date or "Not specified"
    }
    
    # Select and format the appropriate prompt
    selected_prompt = prompt_router.get(category, general_prompt)
    formatted_messages = selected_prompt.format_messages(**prompt_data)
    
    return {
        "analysis": analysis,
        "original_query": data["original_query"],
        "prompt_used": category,
        "messages": formatted_messages
    }

# Create the response generator using LCEL
response_generator = (
    RunnableLambda(route_to_prompt) 
    | RunnableLambda(lambda x: {
        **x,
        "response": llm.invoke(x["messages"])
    })
)

print("✅ Response generator component created with dynamic routing!")

## 5. Component 3: Conversation Summarization & Persistence <a id='component3'></a>

This component creates structured summaries of interactions for logging and analysis.

In [ ]:
def create_conversation_summary(data: Dict[str, Any]) -> Dict[str, Any]:
    """
    Creates a structured summary of the customer service interaction.
    
    Args:
        data: Dictionary containing analysis and response information
        
    Returns:
        Dictionary with complete conversation summary data
    """
    analysis = data["analysis"]
    response = data["response"]
    
    # Extract mentioned products
    mentioned_products = []
    if analysis.entities.product_name:
        mentioned_products.append(analysis.entities.product_name)
    
    # Determine resolution status based on urgency and sentiment
    resolution_status = "resolved"  # Default
    if analysis.urgency_level == "high":
        resolution_status = "escalated"
    elif analysis.customer_sentiment == "negative":
        resolution_status = "pending"
    
    # Determine if follow-up is required
    follow_up_required = (
        analysis.urgency_level == "high" or 
        analysis.customer_sentiment == "negative"
    )
    
    # Create conversation summary
    summary_data = {
        "timestamp": datetime.now().isoformat(),
        "customer_id": "auto_generated",
        "conversation_summary": f"Customer inquired about {analysis.query_category.replace('_', ' ')} with {analysis.customer_sentiment} sentiment",
        "query_category": analysis.query_category,
        "customer_sentiment": analysis.customer_sentiment,
        "urgency_level": analysis.urgency_level,
        "mentioned_products": mentioned_products,
        "extracted_information": {
            "product_name": analysis.entities.product_name,
            "order_number": analysis.entities.order_number,
            "date": analysis.entities.date
        },
        "resolution_status": resolution_status,
        "actions_taken": [
            f"Provided {analysis.query_category.replace('_', ' ')} assistance",
            "Analyzed customer query and sentiment",
            "Generated personalized response"
        ],
        "follow_up_required": follow_up_required,
        "agent_response": response.content
    }
    
    return summary_data

# Create the summary generation prompt
summary_prompt = ChatPromptTemplate.from_messages([
    ("system", """Generate a structured summary of this customer service interaction.
    
    Conversation data:
    {conversation_data}
    
    Create a complete and accurate ConversationSummary."""),
    ("human", "Please generate the conversation summary.")
])

print("✅ Conversation summarization component created!")

summary_generator = summary_prompt | llm_summarizer

## 6. Complete Chain with LCEL <a id='chain'></a>

Now we'll combine all components into a single chain using LangChain Expression Language.

In [ ]:
# Build the complete chain using LCEL
complete_chain = (
    # Step 1: Analyze the query
    RunnablePassthrough.assign(
        analysis=lambda x: query_analyzer.invoke({"query": x["query"]})
    )
    # Step 2: Prepare data for response generation
    | RunnableLambda(lambda x: {
        "original_query": x["query"],
        "analysis": x["analysis"]
    })
    # Step 3: Generate response based on analysis
    | response_generator
    # Step 4: Create conversation summary
    | RunnableLambda(lambda x: {
        **x,
        "summary_data": create_conversation_summary(x)
    })
    # Step 5: Generate final structured summary
    | RunnableLambda(lambda x: {
        "response": x["response"].content,
        "summary": summary_generator.invoke({
            "conversation_data": json.dumps(x["summary_data"], indent=2)
        })
    })
)

# Helper function to process customer queries
def process_customer_query(query: str) -> Dict[str, Any]:
    """
    Process a customer query through the complete chain.
    
    Args:
        query: The customer's query string
        
    Returns:
        Dictionary containing the response and conversation summary
    """
    try:
        result = complete_chain.invoke({"query": query})
        return result
    except Exception as e:
        print(f"❌ Error processing query: {str(e)}")
        return None

print("✅ Complete LCEL chain created successfully!")

## 7. Testing with Sample Queries <a id='testing'></a>

Let's test our complete chain with the suggested test queries.

In [ ]:
# Define test queries as specified in the requirements
test_queries = [
    {
        "name": "Neutral-Informative",
        "query": "Hello, I'd like to know if you have the new iPhone 15 in stock and how much shipping costs to Chicago",
        "expected_category": "product_inquiry"
    },
    {
        "name": "Urgent-Negative",
        "query": "This is an emergency! My order #TEC-2024001 never arrived and I need that laptop for work tomorrow!",
        "expected_category": "billing"
    },
    {
        "name": "Satisfied-Positive",
        "query": "Thank you so much for the excellent service with my previous purchase, I want to buy gaming headphones",
        "expected_category": "product_inquiry"
    },
    {
        "name": "Frustrated-Technical",
        "query": "I can't configure the router I bought last week, I've tried everything and it doesn't work",
        "expected_category": "technical_support"
    },
    {
        "name": "Formal-Billing",
        "query": "Good morning, I need the receipt for my purchase from December 15th, order #TEC-2023089",
        "expected_category": "billing"
    },
    {
        "name": "Warranty-Query",
        "query": "I bought a tablet 8 months ago and now it won't turn on, how do I use the warranty?",
        "expected_category": "returns"
    }
]

print(f"🧪 Ready to test {len(test_queries)} sample queries")

In [ ]:
# Run all test queries and collect results
test_results = []

print("=== RUNNING TEST QUERIES ===")
print("\n" + "="*80 + "\n")

for i, test in enumerate(test_queries, 1):
    print(f"📋 TEST {i}/{len(test_queries)}: {test['name']}")
    print(f"📝 Query: {test['query']}")
    print("-" * 80)
    
    # Process the query
    result = process_customer_query(test['query'])
    
    if result:
        test_results.append(result)
        
        # Display results
        print(f"\n🤖 AGENT RESPONSE:")
        print(f"{result['response']}")
        
        print(f"\n📊 CONVERSATION SUMMARY:")
        summary = result['summary']
        print(f"- Category: {summary.query_category}")
        print(f"- Sentiment: {summary.customer_sentiment}")
        print(f"- Urgency: {summary.urgency_level}")
        print(f"- Status: {summary.resolution_status}")
        print(f"- Follow-up Required: {summary.follow_up_required}")
        
        # Check if category matches expected
        if summary.query_category == test['expected_category']:
            print(f"\n✅ Category classification correct!")
        else:
            print(f"\n⚠️  Category mismatch: Expected '{test['expected_category']}', got '{summary.query_category}'")
        
        # Show extracted information if any
        if summary.mentioned_products:
            print(f"\n📦 Products Mentioned: {', '.join(summary.mentioned_products)}")
        
        extracted_information = summary.extracted_information
        print(f"\n📌 EXTRACTED INFORMATION:")
        print(f"- Product Name: {extracted_information.product_name}")
        print(f"- Order Number: {extracted_information.order_number}")
        print(f"- Date: {extracted_information.date}")
        
    else:
        print("❌ Failed to process query")
    
    print("\n" + "="*80 + "\n")

print(f"✅ Completed testing {len(test_results)}/{len(test_queries)} queries successfully!")

## Summary and Next Steps

### What We've Built:
1. **Query Analyzer**: Classifies queries and extracts entities using structured output
2. **Dynamic Response Generator**: Routes queries to appropriate prompts based on category
3. **Conversation Summarizer**: Creates structured logs of all interactions
4. **Complete LCEL Chain**: Integrates all components seamlessly
5. **LangSmith Integration**: Enables tracing and debugging

### Key Features:
- ✅ Structured output with Pydantic models
- ✅ Dynamic prompt routing based on query category
- ✅ Sentiment and urgency detection
- ✅ Entity extraction (products, order numbers, dates)
- ✅ Automatic resolution status determination
- ✅ Follow-up requirement detection

### Resources:
- [LangChain Documentation](https://python.langchain.com/)
- [LangSmith Documentation](https://docs.smith.langchain.com/)
- [LCEL Guide](https://python.langchain.com/docs/expression_language/)
- [Pydantic Documentation](https://docs.pydantic.dev/)

- https://python.langchain.com/docs/concepts/lcel
- https://www.pinecone.io/learn/series/langchain/langchain-expression-language
- https://python.langchain.com/api_reference/core/runnables.html